In [1]:
import pandas as pd
dataset=pd.read_csv("Social_Network_Ads.csv")
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [2]:
dataset=dataset.drop("User ID",axis=1)
dataset

,Gender,Age,EstimatedSalary,Purchased
0,Male,19,19000,0
1,Male,35,20000,0
2,Female,26,43000,0
3,Female,27,57000,0
4,Male,19,76000,0
...,...,...,...,...
395,Female,46,41000,1
396,Male,51,23000,1
397,Female,50,20000,1
398,Male,36,33000,0


In [3]:
independent=dataset.drop("Purchased",axis=1)
dependent=dataset["Purchased"]

In [4]:
from sklearn.model_selection import train_test_split
Xtrain,Xtest,Ytrain,Ytest=train_test_split(independent,dependent,test_size=0.3,random_state=0)

In [5]:
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
Preprocess=ColumnTransformer(transformers=[
    ('num',StandardScaler(),['Age','EstimatedSalary']),
    ('cat',OneHotEncoder(drop='first'),['Gender']),
])

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
pipe=Pipeline(steps=[
    ('pre',Preprocess),
    ('model',KNeighborsClassifier())
])

In [7]:
from sklearn.model_selection import GridSearchCV
#n_neighborsint, default=5
#weights{‘uniform’, ‘distance’}, callable or None, default=’uniform’
param_grid={'model__n_neighbors':[5,10,15,25,30,40,50,60,100],'model__weights':['uniform','distance']}
grid=GridSearchCV(pipe,param_grid,n_jobs=-1,verbose=3)
grid.fit(Xtrain,Ytrain)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(estimator=Pipeline(steps=[('pre',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['Age',
                                                                          'EstimatedSalary']),
                                                                        ('cat',
                                                                         OneHotEncoder(drop='first'),
                                                                         ['Gender'])])),
                                       ('model', KNeighborsClassifier())]),
             n_jobs=-1,
             param_grid={'model__n_neighbors': [5, 10, 15, 25, 30, 40, 50, 60,
                                                100],
                         'model__weights': ['uniform', 'distance']},
             verbose=3)

In [8]:
grid.best_params_

{'model__n_neighbors': 5, 'model__weights': 'uniform'}

In [9]:
Ypred=grid.predict(Xtest)
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(Ytest,Ypred)
print(cm)

[[73  6]
 [ 4 37]]


In [10]:
from sklearn.metrics import classification_report
clf_report=classification_report(Ytest,Ypred)
print(clf_report)

              precision    recall  f1-score   support

           0       0.95      0.92      0.94        79
           1       0.86      0.90      0.88        41

    accuracy                           0.92       120
   macro avg       0.90      0.91      0.91       120
weighted avg       0.92      0.92      0.92       120



In [11]:
len(Xtrain)

280

In [ ]:
Choosing K Value(range of k)

In [19]:
## No fixed formula for best k in KNN. Common practice: start with small odd values and tune using CV.
## Rule of thumb: try k up to √(n_samples). Best k is chosen where error/accuracy stabilizes.
import math
max_k = int(math.sqrt(len(Xtrain)))
print("max_k:",max_k)
#range(start, stop, step)
print("Range of K:",list(range(1, max_k+1, 2)))
param_grid = {
    'model__n_neighbors': list(range(1, max_k+1, 2)),  # only odd k’s to avoid ties
    'model__weights': ['uniform', 'distance']
}

from sklearn.model_selection import GridSearchCV
grid=GridSearchCV(pipe,param_grid,n_jobs=-1,verbose=3)
grid.fit(Xtrain,Ytrain)

max_k: 16
Range of K: [1, 3, 5, 7, 9, 11, 13, 15]
Fitting 5 folds for each of 16 candidates, totalling 80 fits


GridSearchCV(estimator=Pipeline(steps=[('pre',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['Age',
                                                                          'EstimatedSalary']),
                                                                        ('cat',
                                                                         OneHotEncoder(drop='first'),
                                                                         ['Gender'])])),
                                       ('model', KNeighborsClassifier())]),
             n_jobs=-1,
             param_grid={'model__n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15],
                         'model__weights': ['uniform', 'distance']},
             verbose=3)

In [13]:
grid.best_params_

{'model__n_neighbors': 5, 'model__weights': 'uniform'}

In [14]:
Ypred=grid.predict(Xtest)
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(Ytest,Ypred)
print(cm)

[[73  6]
 [ 4 37]]


In [15]:
from sklearn.metrics import classification_report
clf_report=classification_report(Ytest,Ypred)
print(clf_report)

              precision    recall  f1-score   support

           0       0.95      0.92      0.94        79
           1       0.86      0.90      0.88        41

    accuracy                           0.92       120
   macro avg       0.90      0.91      0.91       120
weighted avg       0.92      0.92      0.92       120

